In [1]:
from neo4j import GraphDatabase
import cypher_utils as ctl
import utils as utl

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "test"))

In [2]:
def cypher(cmd):
    ctl.run(driver,cmd)

In [3]:
json_graph = utl.load_json("Graph_blueprints.json")

In [4]:
len(json_graph["nodes"])

6

- one create command per node, hardcoded uid as name
- usage of create requires a clean first, otherwise use merge

In [7]:
for n in json_graph["nodes"]:
    cmd = f"CREATE (p:{n['_label']} {{name:'{n['name']}' }})"
    print(cmd)
    cypher(cmd)

CREATE (p:Program {name:'lop' })
CREATE (p:Person {name:'vadas' })
CREATE (p:Person {name:'marko' })
CREATE (p:Person {name:'peter' })
CREATE (p:Program {name:'ripple' })
CREATE (p:Person {name:'josh' })


one create command per node, generic properties excluding '_label'

In [5]:
for n in json_graph["nodes"]:
    props = ""
    prop_count = 0
    for key,value in n.items():
        if(key == "_label"):
            continue
        if(prop_count > 0):
            props += ", "
        props += f"{key}:'{n[key]}'"
        prop_count += 1
    cmd = f"CREATE (p:{n['_label']} {{{props}}})"
    print(cmd)
    cypher(cmd)

CREATE (p:Program {name:'lop', lang:'java'})
CREATE (p:Person {name:'vadas', age:'27'})
CREATE (p:Person {name:'marko', age:'29'})
CREATE (p:Person {name:'peter', age:'35'})
CREATE (p:Program {name:'ripple', lang:'java'})
CREATE (p:Person {name:'josh', age:'32'})


Single create command for all nodes, generic properties excluding '_label'

In [9]:
uid = json_graph["edges_nodes_id"]
cmd = "CREATE "
for n in json_graph["nodes"]:
    props = ""
    prop_count = 0
    for key,value in n.items():
        if(key == "_label"):
            continue
        if(prop_count > 0):
            props += ", "
        props += f"{key}:'{n[key]}'"
        prop_count += 1
    cmd += f"\n ({n[uid]}:{n['_label']} {{{props}}}),"
cmd = cmd[:-1]+";"
print(cmd)
cypher(cmd)

CREATE 
 (lop:Program {name:'lop', lang:'java'}),
 (vadas:Person {name:'vadas', age:'27'}),
 (marko:Person {name:'marko', age:'29'}),
 (peter:Person {name:'peter', age:'35'}),
 (ripple:Program {name:'ripple', lang:'java'}),
 (josh:Person {name:'josh', age:'32'});


- creation of edge in pre-existing graph using match
- multiple hits create edge between all cross-matches
- the cypher variables are temporary to the current cmd

In [12]:
cypher('''
    MATCH (n) where n.name = 'josh'
    MATCH (m) where m.name = 'ripple'
    CREATE (n)-[:created {weight:0.4}]->(m)
    ''')

create the whole graph with nodes and edges in one cypher command

In [18]:
uid = json_graph["edges_nodes_id"]
cmd = "CREATE "
for n in json_graph["nodes"]:
    props = ""
    prop_count = 0
    for key,value in n.items():
        if(key == "_label"):
            continue
        if(prop_count > 0):
            props += ", "
        if(type(n[key]) == str):
            props += f"{key}:'{n[key]}'"
        else:
            props += f"{key}:{n[key]}"
        prop_count += 1
    cmd += f"\n ({n[uid]}:{n['_label']} {{{props}}}),"


for e in json_graph["edges"]:
    props = ""
    prop_count = 0
    for key,value in e.items():
        if(key in ["_label","_in","_out"]):
            continue
        if(prop_count > 0):
            props += ", "
        if(type(e[key]) == str):
            props += f"{key}:'{e[key]}'"
        else:
            props += f"{key}:{e[key]}"
        prop_count += 1
    cmd += f"\n ({e['_out']})-[:{e['_label']} {{{props}}}]->({e['_in']}),"
    
cmd = cmd[:-1]+";"
print(cmd)
cypher(cmd)

CREATE 
 (lop:Program {name:'lop', lang:'java'}),
 (vadas:Person {name:'vadas', age:27}),
 (marko:Person {name:'marko', age:29}),
 (peter:Person {name:'peter', age:35}),
 (ripple:Program {name:'ripple', lang:'java'}),
 (josh:Person {name:'josh', age:32}),
 (josh)-[:created {weight:1}]->(ripple),
 (marko)-[:knows {weight:0.5}]->(vadas),
 (marko)-[:created {weight:0.4000000059604645}]->(lop),
 (marko)-[:knows {weight:1}]->(josh),
 (josh)-[:created {weight:0.4000000059604645}]->(lop),
 (peter)-[:created {weight:0.20000000298023224}]->(lop);
